In [1]:
import pickle
from pkg_resources import resource_filename
from coddiwomple.openmm.coddiwomple import endstate_equilibration
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
import os
from simtk import unit

RDKit WARNING: [14:00:43] Enabling RDKit 2019.09.3 jupyter extensions


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


In [ ]:
factory_pickle = f"/data/perses_data/benzene_fluorobenzene.vacuum.factory.pkl"
_pickl = resource_filename('coddiwomple', factory_pickle)
with open(_pickl, 'rb') as f:
    factory = pickle.load(f)


In [ ]:
factory.hybrid_system

In [ ]:
#os.system(f"rm -r tester")

In [ ]:
#vacuum equilibration
endstate_equilibration(system = factory.hybrid_system,
                           endstate_positions = factory.hybrid_positions,
                           box_vectors = factory.hybrid_system.getDefaultPeriodicBoxVectors(),
                           directory_name = 'tester',
                           trajectory_prefix = 'eq',
                           md_topology = factory.hybrid_topology,
                           number_of_applications = 10,
                           steps_per_application = 500,
                           endstate_parameters = 0.0,
                           alchemical_composability = RelativeAlchemicalState)

In [2]:
factory_pickle = f"/data/perses_data/benzene_fluorobenzene.solvent.factory.pkl"
_pickl = resource_filename('coddiwomple', factory_pickle)
with open(_pickl, 'rb') as f:
    factory = pickle.load(f)
    
#remove the barostat since we are living in NVT world
force_names = {force.__class__.__name__ : index for index, force in enumerate(factory.hybrid_system.getForces())}
barostat_index = force_names['MonteCarloBarostat']
print(force_names)


{'MonteCarloBarostat': 0, 'CustomBondForce': 9, 'HarmonicBondForce': 2, 'CustomAngleForce': 3, 'HarmonicAngleForce': 4, 'CustomTorsionForce': 5, 'PeriodicTorsionForce': 6, 'NonbondedForce': 7, 'CustomNonbondedForce': 8}


In [3]:
factory.hybrid_system.removeForce(barostat_index)

In [ ]:
#solvent equilibration
endstate_equilibration(system = factory.hybrid_system,
                           endstate_positions = factory.hybrid_positions,
                           box_vectors = factory.hybrid_system.getDefaultPeriodicBoxVectors(),
                           directory_name = 'tester',
                           trajectory_prefix = 'eq_2_solvent',
                           md_topology = factory.hybrid_topology,
                           number_of_applications = 10,
                           steps_per_application = 500,
                           endstate_parameters = 0.0,
                           alchemical_composability = RelativeAlchemicalState)

In [ ]:
#let's open the equilibrated solvent model to make sure it is actually working
import mdtraj as md
traj = md.Trajectory.load(f"{os.getcwd()}/tester/eq_2_solvent.0000.pdb")

In [ ]:
traj.n_frames

In [ ]:
#looks like it worked...

In [4]:
x = 'fractional_iteration'
alchemical_functions = {
                         'lambda_sterics_core': x,
                         'lambda_electrostatics_core': x,
                         'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                         'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                         'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                         'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                         'lambda_bonds': x,
                         'lambda_angles': x,
                         'lambda_torsions': x}

In [6]:
#let's try to anneal some of the solvent samples
from coddiwomple.openmm.coddiwomple import annealed_importance_sampling
annealed_importance_sampling(system = factory.hybrid_system,
                                 endstate_cache_filename = f"{os.getcwd()}/tester/eq_2_solvent.0000.pdb",
                                 directory_name = 'tester',
                                 trajectory_prefix = 'neq',
                                 md_topology = factory.hybrid_topology,
                                 alchemical_functions = alchemical_functions,
                                 number_of_applications = 5,
                                 steps_per_application = 100,
                                 endstate_parameters = 0.0,
                                 alchemical_composability = RelativeAlchemicalState,
                                 integrator_kwargs = {'temperature': 300.0 * unit.kelvin,
                                                      'collision_rate': 1.0 / unit.picoseconds,
                                                      'timestep': 2.0 * unit.femtoseconds,
                                                      'splitting': "P I H N S V R O R V",
                                                      'constraint_tolerance': 1e-5},
                                record_state_work_interval = 1,
                                trajectory_write_interval = 1)

DEBUG:openmm_states:successfully instantiated OpenMMPDFState equipped with the following parameters: {'lambda_sterics_delete': 0.0, 'lambda_electrostatics': None, 'lambda_torsions': 0.0, 'lambda_bonds': 0.0, 'lambda_electrostatics_insert': 0.0, 'lambda_angles': 0.0, 'lambda_electrostatics_delete': 0.0, 'lambda_electrostatics_core': 0.0, 'lambda_sterics_insert': 0.0, 'lambda_sterics_core': 0.0, 'lambda_sterics': None}
DEBUG:openmm_states:successfully updated OpenMMPDFState parameters as follows: {'lambda_sterics_delete': 0.0, 'lambda_torsions': 0.0, 'lambda_bonds': 0.0, 'lambda_electrostatics_insert': 0.0, 'lambda_angles': 0.0, 'lambda_electrostatics_delete': 0.0, 'lambda_electrostatics_core': 0.0, 'lambda_sterics_insert': 0.0, 'lambda_sterics_core': 0.0}
DEBUG:openmm_reporters:creating trajectory storage object...
DEBUG:openmm_propagators:initializing OMMLIAIS...
DEBUG:openmm_propagators:OMMLIAIS: metropolization is False
DEBUG:openmm_propagators:OMMLIAIS: successfully parsed splitting

step      0 : allow forces to update the context state
step      1 : if(has_kT_changed = 1):
step      2 :    sigma <- sqrt(kT/m)
step      3 :    has_kT_changed <- 0
step      4 : end
step      5 : old_pe <- energy
step      6 : iteration <- iteration + 1.0
step      7 : fractional_iteration <- iteration / niterations
step      8 : lambda_sterics_core <- fractional_iteration
step      9 : lambda_electrostatics_core <- fractional_iteration
step     10 : lambda_sterics_insert <- select(step(fractional_iteration - 0.5), 1.0, 2.0 * fractional_iteration)
step     11 : lambda_sterics_delete <- select(step(fractional_iteration - 0.5), 2.0 * (fractional_iteration - 0.5), 0.0)
step     12 : lambda_electrostatics_insert <- select(step(fractional_iteration - 0.5), 2.0 * (fractional_iteration - 0.5), 0.0)
step     13 : lambda_electrostatics_delete <- select(step(fractional_iteration - 0.5), 1.0, 2.0 * fractional_iteration)
step     14 : lambda_bonds <- fractional_iteration
step     15 : lambda_an

Exception ignored in: <bound method PDBTrajectoryFile.__del__ of <mdtraj.formats.pdb.pdbfile.PDBTrajectoryFile object at 0x7f9fac11ec50>>
Traceback (most recent call last):
  File "/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/mdtraj/formats/pdb/pdbfile.py", line 666, in __del__
    self.close()
  File "/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/mdtraj/formats/pdb/pdbfile.py", line 501, in close
    self._write_footer()
  File "/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/mdtraj/formats/pdb/pdbfile.py", line 429, in _write_footer
    index1 = atomIndex[atom1]
KeyError: (MOL1-C1,)
DEBUG:openmm_propagators:	context_parameters after integration: {'lambda_angles': 1.0, 'lambda_bonds': 1.0, 'lambda_electrostatics_core': 1.0, 'lambda_electrostatics_delete': 1.0, 'lambda_electrostatics_insert': 1.0, 'lambda_sterics_core': 1.0, 'lambda_sterics_delete': 1.0, 'lambda_sterics_insert': 1.0, 'lambda_torsions': 1.0, 'softcore_alpha': 0.85}
 2

{0: [0.0,
  0.03789190806618788,
  0.0745245308961151,
  0.11027135141047599,
  0.14560269574727416,
  0.18086627350153942,
  0.21645551178002168,
  0.25305580263879696,
  0.29117343122824035,
  0.33082844535082184,
  0.37166843526836674,
  0.41351988377625154,
  0.456617225894998,
  0.5016493138995255,
  0.5491692403359992,
  0.5992596981548745,
  0.6514918181596001,
  0.704921403762292,
  0.758304923456589,
  0.8104805953678889,
  0.8610269545499295,
  0.9103258941973598,
  0.9591907537351746,
  1.008259831241545,
  1.0579217285837128,
  1.1084817466394903,
  1.1600832109947277,
  1.212671961137365,
  1.2659593961806952,
  1.3195838109636788,
  1.3733155139881532,
  1.4270859234377036,
  1.4806078555749977,
  1.5335316449986045,
  1.5854185951097903,
  1.6361663190967641,
  1.6861948551828676,
  1.7361060067908496,
  1.786527486600627,
  1.8381611487340355,
  1.8916536060688205,
  1.9477867795637465,
  2.0073254486409846,
  2.0706049099084463,
  2.137439649094165,
  2.207417972697535

In [ ]:
traj = md.Trajectory.load(os.path.join(os.getcwd(), 'test_fluoro', 'solvent.neq.pdb'))

    
hybrid_system = factory.hybrid_system
num_frames = traj.n_frames
pdf_state = OpenMMPDFState(system = factory.hybrid_system, alchemical_composability = RelativeAlchemicalState, pressure=None)

In [ ]:
import copy
from coddiwomple.openmm.integrators import OMMLIAIS
from coddiwomple.openmm.propagators import OMMAISP

In [ ]:
n_steps = 100
x = 'fractional_iteration'
alchemical_functions = {
                         'lambda_sterics_core': x,
                         'lambda_electrostatics_core': x,
                         'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                         'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                         'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                         'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                         'lambda_bonds': x,
                         'lambda_angles': x,
                         'lambda_torsions': x}

ais_integrator = OMMLIAIS(
                         alchemical_functions,
                         n_steps,
                         temperature=300.0 * unit.kelvin,
                         collision_rate=1.0 / unit.picoseconds,
                         timestep=1.0 * unit.femtoseconds,
                         splitting="P I H N S V R O R V",
                         constraint_tolerance=1e-6)



try:
    os.system(f"rm -r fluoro_bounce")
except:
    pass

reporter = OpenMMReporter('fluoro_bounce', 'runner', factory.hybrid_topology)
ais_propagator = OMMAISPR(openmm_pdf_state = pdf_state, integrator = ais_integrator, record_state_work_interval = 1, reassign_velocities=True, reporter=reporter)

master_works = []
frames = [33, 45, 10, 34, 100, 101, 54, 76]
import tqdm
for num_trajectories in tqdm.trange(7):
    positions = traj.xyz[frames[num_trajectories]] * unit.nanometers
    particle_state = OpenMMParticleState(positions=positions, box_vectors=traj.unitcell_vectors[frames[num_trajectories]]*unit.nanometers)

    _, return_dict = ais_propagator.apply(particle_state, n_steps=n_steps, apply_pdf_to_context = True, reset_integrator = True)

so it looks like there is a problem in the way that the initial energy of the non_initialized state is running...



In [ ]:
ais_propagator._record_state_work_interval

In [ ]:
ais_propagator._state_works

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
for q in np.array(master_works):
    print(len(q))
    plt.plot(np.append(np.array([0.]), q))

In [ ]:
from openmmtools.states import *
from openmmtools.integrators import LangevinIntegrator
import openmmtools.cache
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
thermostate = ThermodynamicState(hybrid_system, temperature=300*unit.kelvin, pressure=None)
alchemical_state = RelativeAlchemicalState.from_system(hybrid_system)
comp = CompoundThermodynamicState(thermostate, [alchemical_state])

integrator = LangevinIntegrator(temperature=thermostate.temperature)
context = comp.create_context(integrator)

positions1, positions2 = traj.xyz[33] * unit.nanometers, traj.xyz[45] * unit.nanometers
sampler_state1 = SamplerState(positions1, box_vectors = traj.unitcell_vectors[33]*unit.nanometers)
sampler_state2 = SamplerState(positions2, box_vectors = traj.unitcell_vectors[45]*unit.nanometers)



In [ ]:
n_steps = 10
x = 'fractional_iteration'
alchemical_functions = {
                         'lambda_sterics_core': x,
                         'lambda_electrostatics_core': x,
                         'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                         'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                         'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                         'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                         'lambda_bonds': x,
                         'lambda_angles': x,
                         'lambda_torsions': x}


AIS_integrator = OpenMMIAAISLangevinIntegrator(openmm_pdf_update_functions = alchemical_functions,
                                                n_steps = n_steps,
                                                temperature=pdf_state.temperature,
                                                collision_rate=1.0 / unit.picoseconds,
                                                timestep=4.0 * unit.femtoseconds,
                                                splitting="P I H N S V R O R V",
                                                constraint_tolerance=1e-6)



try:
    os.system(f"rm -r test_fluoro_neq")
except:
    pass

ais_propagator = OpenMMBaseIntegrationPropagator(openmm_pdf_state = pdf_state, integrator = AIS_integrator)
master_works = []
frames = [45, 33]
for num_trajectories in range(2):
    ais_propagator.integrator.reset()
    num_global_vars = ais_propagator.integrator.getNumGlobalVariables()
    global_vars = {ais_propagator.integrator.getGlobalVariableName(q): ais_propagator.integrator.getGlobalVariable(q) for q in range(num_global_vars)}
    print(global_vars)
    random_frame = frames[num_trajectories]
    positions = traj.xyz[random_frame] * unit.nanometers
    particle_state = OpenMMParticleState(positions=positions, box_vectors=traj.unitcell_vectors[random_frame]*unit.nanometers)
    particle_state.apply_to_context(ais_propagator.context)
    #pdf_state.apply_to_context(ais_propagator.context)
    context_params = ais_propagator.context.getParameters()
    context_params_nonswg = {q: context_params[q] for q in context_params}
    print(f"context_params: ",context_params_nonswg)
    #[ais_propagator.context.setGlobalVariableByName(key, 0.0) for key in global_vars.keys()]
    import tqdm
    works = []
    for _ in tqdm.trange(n_steps):
        _, return_dict = ais_propagator.apply(particle_state, n_steps=1, return_state_work=True, return_proposal_work=True,reset_integrator=False)
        num_global_vars = ais_propagator.integrator.getNumGlobalVariables()
        global_vars = {ais_propagator.integrator.getGlobalVariableName(q): ais_propagator.integrator.getGlobalVariable(q) for q in range(num_global_vars)}
        print(f"\titeration: {_}:")
        for key, item in global_vars.items():
            print(f"\t\t{key}: {item}")
        #reporter.record([particle])
        context_state = ais_propagator.context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                 enforcePeriodicBox=ais_propagator.pdf_state.is_periodic)
        pe  = context_state.getPotentialEnergy().value_in_unit(_OPENMM_ENERGY_UNIT)
        print(f"ENERGY_during: {pe}")
        works.append(return_dict['state_work'])
#     try:
#         reporter.record([particle], save_to_disk=True)
#     except Exception as e:
#         print(e)
    master_works.append(works)